In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from numpy import random
from datetime import datetime
from math import nan
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
import statsmodels.api as sm

%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)

In [3]:
sns.set_style("whitegrid")

### 1. Import and clean data preliminarily

In [ ]:
matches_combined = pd.read_csv('../dataset/matches_combines_2.csv')
matches = pd.read_csv('../dataset/matches.csv')
player = pd.read_csv('../dataset/player.csv')
player_detail = pd.read_csv('../dataset/player_detail.csv')

In [ ]:
matches_combined.shape

In [ ]:
matches_combined.head()

In [ ]:
matches_combined['result'].value_counts()

- Home win has more observations than home lose

In [ ]:
#drop columns that have less significant influence on match result by common sense
columns = matches_combined.columns
new_columns = [col for col in columns if 'dribbling_GK' not in col
                                  and 'sprint_speed_GK' not in col
                                  and 'passing_speed_GK' not in col
                                  and 'shooting_GK' not in col 
                                  and 'defence_GK' not in col
                                  and 'passing_GK' not in col
                                  and 'dribbling_CB' not in col
                                  and 'sprint_speed_CB' not in col
                                  and 'passing_speed_CB' not in col
                                  and 'shooting_CB' not in col   
                                  and 'defence_ST' not in col
                                  and 'defence_W' not in col
                                  and 'sprint_speed_MF' not in col
                                  and 'goalkeeping_CB' not in col
                                  and 'goalkeeping_MF' not in col
                                  and 'goalkeeping_SB' not in col
                                  and 'goalkeeping_ST' not in col
                                  and 'goalkeeping_W' not in col]

matches_combined = matches_combined[new_columns]

In [ ]:
matches_combined.isna().sum()

- Missing data appear on position 'W' and 'SB'.

In [ ]:
matches_combined.dropna(inplace = True)
matches_combined = matches_combined.reset_index(drop = True)
matches_combined.shape

### 2. Check out outliers

In [ ]:
matches_new = matches_combined.drop('match_api_id', axis = 1)
y = matches_new['result']
X = matches_new.drop(['result'], axis = 1)

In [ ]:
# A quick view of all columns distributions
X.hist(bins=20, figsize=(15, 15))
plt.show()

In [ ]:
X.describe()

- A few parameters look way below average. 

In [ ]:
z_scores = np.abs((X - X.mean()) / X.std())
outliers = z_scores > 4
outliers_rows = matches_combined[outliers.any(axis=1)]

outliers_rows.shape

In [ ]:
# Check which columns have the most outliers
outlier_counts = outliers.sum()
plt.figure(figsize=(8, 4))
outlier_counts.plot(kind='bar')
plt.xlabel('Columns')
plt.ylabel('Number of Outliers')
plt.title('Number of Outliers in Each Column')
plt.show()

In [ ]:
outliers_rows.head(30)

In [ ]:
# Check the match which shooting_ST_home = 12
matches.query('match_api_id == 483333')

In [ ]:
player.query('player_api_id == 37254')

In [ ]:
player_detail.query('player_api_id == 37254')

- It seems that this player had good days and bad days

In [ ]:
# Check the match which goalkeeping_GK_home = 16.3
matches.query('match_api_id == 654326')

In [ ]:
player.query('player_api_id == 94289')

A quick search shows this player plays midfielder regularly instead of goalkeeper.

### 2. Check out correlations

In [ ]:
# Plot correlation heatmap
plt.subplots(figsize=(12,12))
sns.heatmap(matches_new.iloc[:, :20].corr(),annot=True, cmap=sns.diverging_palette(220, 20, as_cmap=True))

#### Observations: 
#### - there are constant high correlations between 'passing' and 'dribbling' of one position
Decision: remove 'drbbling' for all positions
#### - there's a high correlation between 'shooting_MF' and 'passing_MF'
Decision: remove 'shooting_MF'
#### - there are constant high correlations between different position which results from unclear reason. 
Decision: ignore

In [ ]:
# Drop features
matches_new = matches_new.drop(['dribbling_MF_home', 'dribbling_SB_home', 'dribbling_ST_home',
       'dribbling_W_home', 'dribbling_MF_away', 'dribbling_SB_away', 'dribbling_ST_away', 'dribbling_W_away',
      'shooting_MF_home', 'shooting_MF_away'], axis = 1)

### 3. Quick view on logistic modeling insights using statsmodel

In [ ]:
y = matches_new['result']
X = matches_new.drop(['result'], axis = 1)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X)

In [ ]:
X_transformed = scaler.transform(X)
transformed_df = pd.DataFrame(X_transformed, columns = X.columns)
transformed_df['result'] = matches_new['result']

In [ ]:
transformed_df.head()

In [ ]:
X_sm = sm.add_constant(X)

label_mapping = {'lose':0, 'tie':0.5, 'win':1}

y_sm = y.map(label_mapping)
log_reg = sm.Logit(y_sm, X_sm).fit()
print(log_reg.summary2())

#### Observation: 
#### - defence_MF_home has a negative coef., while defence_MF_away has a positive coef and P = 0.57. 
Decision: remove both 'defence_MF_home' and 'defence_MF_away'

In [ ]:
# Drop features
matches_new = matches_new.drop(['defence_MF_home','defence_MF_away'], axis = 1)

In [ ]:
y = matches_new['result']
X = matches_new.drop(['result'], axis = 1)
scaler = MinMaxScaler()
scaler.fit(X)

X_transformed = scaler.transform(X)

#Create dataframe for visualization
transformed_df = pd.DataFrame(X_transformed, columns = X.columns)
transformed_df['result'] = matches_new['result']

In [ ]:
X_sm = sm.add_constant(X)

label_mapping = {'lose':0, 'tie':0.5, 'win':1}

y_sm = y.map(label_mapping)
log_reg = sm.Logit(y_sm, X_sm).fit()
print(log_reg.summary2())

- Now all home team features have positive coef. and all away team features have negative coef.

### 4. Visualizae high dimensional data using PCA

In [ ]:
for i in range(2,5):
    pca = PCA(n_components=i)
    X_pca_new = pca.fit_transform(X_transformed)
    print('n_components='+str(i))
    print(pca.explained_variance_ratio_)

In [ ]:
pca = PCA(n_components=2)
X_pca_new = pca.fit_transform(X_transformed)

In [ ]:
pca_df = pd.DataFrame(X_pca_new, columns = ['PC1', 'PC2'])
pca_df['result'] = y
pca_df.head()

In [ ]:
fig, axes = plt.subplots(figsize = (10,10))
results = ['win', 'tie', 'lose']
colors = ['lightblue', 'burlywood', 'sienna']

for result, color in zip(results, colors):
    indicesToKeep = pca_df['result'] == result
    axes.scatter(pca_df.loc[indicesToKeep, 'PC1'],
                 pca_df.loc[indicesToKeep, 'PC2'],
                 c=color,
                 label=result,
                 s=2)

axes.set_xlabel('PC1')
axes.set_ylabel('PC2')
axes.legend()
axes.grid()
plt.show()

- Not ideal, but can observe some patterns

In [ ]:
pca_3 = PCA(n_components=3)
X_pca_new_3 = pca_3.fit_transform(X_transformed)
pca_df_3 = pd.DataFrame(X_pca_new_3, columns = ['PC1', 'PC2', 'PC3'])
pca_df_3['result'] = y

In [ ]:
plt.rcParams['figure.figsize'] = (20, 20)
fig = plt.figure()
axes = Axes3D(fig)


results = ['win', 'tie', 'lose']
colors = ['lightblue', 'palegreen', 'sienna']

for result, color in zip(results, colors):
    indicesToKeep = pca_df_3['result'] == result
    axes.scatter(pca_df_3.loc[indicesToKeep, 'PC1'],
                 pca_df_3.loc[indicesToKeep, 'PC2'],
                 pca_df_3.loc[indicesToKeep, 'PC3'],
                 c=color,
                 label=result,
                 s=1)

- Not ideal, but can observe some patterns

In [ ]:
# Save final dataframe
matches_new.to_csv('../dataset/matches_final.csv', index = False)